In [2]:
!pip3 install scikit-learn pandas numpy


Defaulting to user installation because normal site-packages is not writeable


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load the dataset
train_data = pd.read_csv('/Users/bugruster/Developer/PRJSE/KDDTrain+.csv')
test_data = pd.read_csv('/Users/bugruster/Developer/PRJSE/KDDTest+.csv')

# Column names for the dataset
columns = [
    "duration", "protocol_type", "service", "flag", "src_bytes", "dst_bytes", "land", 
    "wrong_fragment", "urgent", "hot", "num_failed_logins", "logged_in", "num_compromised", 
    "root_shell", "su_attempted", "num_root", "num_file_creations", "num_shells", 
    "num_access_files", "num_outbound_cmds", "is_host_login", "is_guest_login", 
    "count", "srv_count", "serror_rate", "srv_serror_rate", "rerror_rate", 
    "srv_rerror_rate", "same_srv_rate", "diff_srv_rate", "srv_diff_host_rate", 
    "dst_host_count", "dst_host_srv_count", "dst_host_same_srv_rate", 
    "dst_host_diff_srv_rate", "dst_host_same_src_port_rate", 
    "dst_host_srv_diff_host_rate", "dst_host_serror_rate", "dst_host_srv_serror_rate", 
    "dst_host_rerror_rate", "dst_host_srv_rerror_rate", "label", "difficulty"
]

train_data.columns = columns
test_data.columns = columns

# Separate features and labels
X_train = train_data.drop(columns=['label', 'difficulty'])
y_train = train_data['label']

X_test = test_data.drop(columns=['label', 'difficulty'])
y_test = test_data['label']

# Convert categorical features into numerical form using LabelEncoder
categorical_cols = ['protocol_type', 'service', 'flag']

for col in categorical_cols:
    encoder = LabelEncoder()
    X_train[col] = encoder.fit_transform(X_train[col])
    X_test[col] = encoder.transform(X_test[col])

# Convert labels to binary (normal vs attack)
y_train = y_train.apply(lambda x: 0 if x == 'normal' else 1)
y_test = y_test.apply(lambda x: 0 if x == 'normal' else 1)


In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pickle

# Split the data (you can also use the entire dataset without splitting if needed)
X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Initialize the RandomForest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train_split, y_train_split)

# Validate the model
y_val_pred = rf_model.predict(X_val)
print(f"Validation Accuracy: {accuracy_score(y_val, y_val_pred)}")

# Evaluate on test data
y_test_pred = rf_model.predict(X_test)
print(f"Test Accuracy: {accuracy_score(y_test, y_test_pred)}")

# Save the trained model to a pickle file
with open('models/rf_model.pkl', 'wb') as model_file:
    pickle.dump(rf_model, model_file)


Validation Accuracy: 0.9986108354832308
Test Accuracy: 0.7758406530032828


In [5]:
with open('models/rf_model.pkl', 'wb') as model_file:
    pickle.dump(rf_model, model_file)
